In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# feature engineering ideas:
# date: encode time of day, weekend/weekday, holiday, amount, etc.
# amount: consider encoding whole numbers

# how to test features:
# compare overall accuracy

### Week 4 Work

In [2]:
data = pd.read_parquet("../data/DSC180B.parquet")
data['dt'] = pd.to_datetime(data['transaction_date'])
data = data[["amount", "dt", "new_category"]]
data

,amount,dt,new_category
8,15.25,2018-03-06,Entertainment
39,16.04,2018-09-29,General Merchandise
45,10.36,2018-10-17,Automotive
52,4.63,2017-07-11,Automotive
55,223.00,2018-09-10,General Merchandise
...,...,...,...
1999981,24.24,2021-02-19,Automotive
1999982,120.60,2021-01-19,General Merchandise
1999983,6.47,2022-02-25,Food and Beverages
1999991,44.98,2021-01-11,General Merchandise


In [4]:
data.dtypes

amount                 float64
dt              datetime64[ns]
new_category            object
dtype: object

In [5]:
data['new_category'].unique()

array(['Entertainment', 'General Merchandise', 'Automotive',
       'Food and Beverages', 'Travel', 'Groceries', 'Healthcare/Medical',
       'Pets/Pet Care'], dtype=object)

In [19]:
data['new_category'].value_counts()

Food and Beverages     162009
General Merchandise    138971
Automotive              63617
Groceries               63541
Entertainment           31492
Travel                  25491
Healthcare/Medical       8392
Pets/Pet Care            2588
Name: new_category, dtype: int64

In [6]:
# step 1: try to make a model without the text column

In [7]:
data['year'] = data['dt'].dt.year
data['quarter'] = data['dt'].dt.quarter
data['month'] = data['dt'].dt.month
data['week'] = data['dt'].dt.week
data['day_of_week'] = data['dt'].dt.day_of_week
data['day_of_month'] = data['dt'].dt.day
data['day_of_year'] = data['dt'].dt.day_of_year
data['is_whole_number'] = data['amount'].apply(lambda x: 1 if str(x).split(".")[1] == "0" else 0)

/var/folders/bg/tjm2hhbn7vl_j003610q1dnc0000gn/T/ipykernel_38802/1333840826.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['week'] = data['dt'].dt.week


In [8]:
X = data[["amount", "is_whole_number", "year", "quarter", "month", "week", "day_of_week", "day_of_month", "day_of_year"]]
y = data[["new_category"]]
display(X)
display(y)
X.dtypes, y.dtypes

,amount,is_whole_number,year,quarter,month,week,day_of_week,day_of_month,day_of_year
8,15.25,0,2018,1,3,10,1,6,65
39,16.04,0,2018,3,9,39,5,29,272
45,10.36,0,2018,4,10,42,2,17,290
52,4.63,0,2017,3,7,28,1,11,192
55,223.00,1,2018,3,9,37,0,10,253
...,...,...,...,...,...,...,...,...,...
1999981,24.24,0,2021,1,2,7,4,19,50
1999982,120.60,0,2021,1,1,3,1,19,19
1999983,6.47,0,2022,1,2,8,4,25,56
1999991,44.98,0,2021,1,1,2,0,11,11


,new_category
8,Entertainment
39,General Merchandise
45,Automotive
52,Automotive
55,General Merchandise
...,...
1999981,Automotive
1999982,General Merchandise
1999983,Food and Beverages
1999991,General Merchandise


(amount             float64
 is_whole_number      int64
 year                 int64
 quarter              int64
 month                int64
 week                 int64
 day_of_week          int64
 day_of_month         int64
 day_of_year          int64
 dtype: object,
 new_category    object
 dtype: object)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model = LogisticRegression(multi_class = "multinomial").fit(X_train, y_train)
preds = model.predict(X_test)

/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kylenero/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
y_test = np.array(y_test['new_category'])
accuracy = (preds == y_test).mean()
accuracy

0.33094909415199675

In [49]:
# step 2: try to make a model with step 1 and kooshas tf-idf (koosha will send tf-idf notebook)

In [50]:
# need to combine eventually